Automatically delint notebooks on save?

delint notebooks?


In [1]:
// TODO: display a tree of require dependencies and make sure it all still compiles properly
var importer = require('../Core');
var fs = require('fs');
var path = require('path');
var {ncp} = require('ncp');
var {execCmd, diffTwoTexts} = importer.import([
    'spawn child process',
    'diff code blocks',
]);
var cell = importer.interpret('diff code blocks');
ncp.limit = 16;

var webstormWin = 'C:\\Program Files\\JetBrains\\WebStorm 2017.1.1\\bin\\format.bat';
var webstormMac = '/Applications/WebStorm.app/Contents/bin/format.sh';
var webstorm = process.platform === 'win32' ? webstormWin : webstormMac;

// TODO: how to force CLI to use project based settings instead of user settings?
var idea = path.resolve(path.join(__dirname, '../.idea'));

function delint(project) {
    return new Promise((resolve, reject) => {
        // output .idea to set the framework preferences?
        return ncp(idea, project + '/.idea', function (err) {
            if (err) {
                return reject(err);
            }
            resolve();
        });
    })
    // TODO: make this a test script for exporting all notebooks on file change watcher
        .then(() => {
            // I think I overwrote the default, some way to env variables for this?
            const cmd = '"' + webstorm + '" "' + project + '"';
            if (!fs.existsSync(project)) {
                fs.mkdirSync(project);
            }
            return execCmd(cmd, {cwd: project})
        })
};
module.exports = delint;



[Function: delint]

test de-linting service?


In [1]:
var fs = require('fs');
var importer = require('../Core');
var delint = importer.import('delint notebooks');
var exportAndDeploy = importer.import('export deploy notebook');
var exportNotebook = importer.import('notebook.ipynb[export cells modules]');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = path.resolve(path.join(__dirname, '../Utilities/.modules'));

// just do it in load mode
if(typeof describe === 'undefined') {
    var it = ((l, func) => func());
    it.timeout = () => {};
    var beforeEach = ((func) => func());
    var describe = ((l, func) => func());
//    var describe = (() => {});
}

describe('de-linting service', () => {
    it('should de-lint a file', () => {
        const r = importer.interpret('diff code blocks');
        assert(r.filename.indexOf('diff.ipynb') > -1);
        return exportAndDeploy(['delint notebooks'])
            .then(() => exportNotebook(__filename))
            .then(() => delint(project))
            .then(r => {
                //console.log(r);
                //var filename = path.join(project, 'Utilities/diff/cell-' + cell.to + '.js');
                //return diffTwoTexts(cell.code, fs.readFileSync(filename).toString());
            })
            .catch(r => console.log(r))
    }).timeout(30000);
})



reading notebook /Users/briancullinan/jupytangular2/Core/interpret all notebooks.ipynb
compiling /Users/briancullinan/jupytangular2/Core/interpret all notebooks.ipynb[0]
reading notebook /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[0]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[1]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[2]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[3]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[4]
compiling /Users/briancullinan/jupytangular2/Core/interpret all notebooks.ipynb[1]
reading notebook /Users/briancullinan/jupytangular2/Utilities/file system.ipynb
compiling /Users/briancullinan/jupytangular2/Utilities/file system.ipynb[5]
reading notebook /Users/briancullinan/jupytangular2/Analytics/budgeting.ipynb
reading notebook /Users/briancullin

TypeError: Cannot read property 'timeout' of undefined

In [2]:
var importer = require('../Core');
var delint = importer.import('delint notebooks');

if(typeof $$ !== 'undefined') {
    var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
    var project = path.resolve(path.join(PROFILE_PATH, 'Documents/subscription.services.eloqua'));
    
    $$.async();
    delint(project)
        .catch(r => console.log(r))
        .then(r => {
            console.log(r);
            $$.done();
        })
}

reading notebook C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\de-linting.ipynb
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\de-linting.ipynb[0]
spawning process "C:\Program Files\JetBrains\WebStorm 2017.1.1\bin\format.bat" "C:\Users\brian.cullinan\Documents\subscription.services.eloqua/"


Jan 04, 2018 11:11:43 AM java.util.prefs.WindowsPreferences <init>



WebStorm 2017.2.5, build WS-172.4343.25 Formatter

Formatting C:/Users/brian.cullinan/Documents/subscription.services.eloqua/.gitignore...
Skipped, not supported.

Formatting C:/Users/brian.cullinan/Documents/subscription.services.eloqua/aws-entry-point.js...
OK

Formatting C:/Users/brian.cullinan/Documents/subscription.services.eloqua/bundle.js...
OK

Formatting C:/Users/brian.cullinan/Documents/subscription.services.eloqua/calculate-price.js...
OK

Formatting C:/Users/brian.cullinan/Documents/subscription.services.eloqua/calculate-price.spec.js...
OK

Formatting C:/Users/brian.cullinan/Documents/subscription.services.eloqua/eloqua-import-create-template.js...
OK

Formatting C:/Users/brian.cullinan/Documents/subscription.services.eloqua/eloqua-import-service.js...
OK

Formatting C:/Users/brian.cullinan/Documents/subscription.services.eloqua/eloqua-import.spec.js...
OK

Formatting C:/Users/brian.cullinan/Documents/subscription.services.eloqua/eloqua_production.json...
OK

Formatting C:

Autofix some linting errors using bslint?

